In [1]:
!nvidia-smi

Thu Jul 22 21:17:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 39.4 MB/s 
     |████████████████████████████████| 3.3 MB 51.2 MB/s 
     |████████████████████████████████| 636 kB 41.5 MB/s 
     |████████████████████████████████| 895 kB 66.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import requests
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.utils import class_weight
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as I
from torch.autograd import Variable
from transformers import AutoModel, BertTokenizerFast
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, BertModel

### Load data

In [5]:
data_name = 'user_reviews.csv'
path = F"/content/gdrive/My Drive/{data_name}"

reviews_df = pd.read_csv(path,lineterminator='\n')
#reviews_df = reviews_df[:5000]
reviews_df.head(5)

,userID,id,date_posted,review,recommended,year,month
0,0,1250,November 5 2011,Simple yet with great replayability. In my opi...,True,2011,November
1,0,22200,July 15 2011,Its unique and worth a playthrough.,True,2011,July
2,0,43110,April 21 2011,Great atmosphere. The gunplay can be a bit chu...,True,2011,April
3,1,251610,June 24 2014,I know what you think when you see this title ...,True,2014,June
4,1,227300,September 8 2013,For a simple (its actually not all that simple...,True,2013,September


In [6]:
reviews_df.tail(5)

,userID,id,date_posted,review,recommended,year,month
58426,25456,70,July 10,a must have classic from steam definitely wort...,True,-1,-1
58427,25456,362890,July 8,this game is a perfect remake of the original ...,True,-1,-1
58428,25457,273110,July 3,had so much fun plaing this and collecting res...,True,-1,-1
58429,25457,730,July 20,:D,True,-1,-1
58430,25457,440,July 2,so much fun :D,True,-1,-1


In [7]:
reviews_df.isnull().sum()

userID          0
id              0
date_posted     0
review         30
recommended     0
year            0
month           0
dtype: int64

In [8]:
reviews_df.dropna(inplace = True)
reviews_df.isnull().sum()

userID         0
id             0
date_posted    0
review         0
recommended    0
year           0
month          0
dtype: int64

In [9]:
print("The length of the dataset:",len(reviews_df))

The length of the dataset: 58401


In [10]:
reviews_data = reviews_df[["review","recommended"]]
reviews_data["recommended"].value_counts()

True     51665
False     6736
Name: recommended, dtype: int64

In [11]:
labels = reviews_data["recommended"].unique()
label_dict = {}
for idx, labl in enumerate(labels):
    label_dict[labl] = idx

In [12]:
reviews_data["label"] = reviews_data["recommended"].replace(label_dict)
reviews_data.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,review,recommended,label
0,Simple yet with great replayability. In my opi...,True,0
1,Its unique and worth a playthrough.,True,0
2,Great atmosphere. The gunplay can be a bit chu...,True,0
3,I know what you think when you see this title ...,True,0
4,For a simple (its actually not all that simple...,True,0


In [13]:
reviews_data["label"].value_counts()

0    51665
1     6736
Name: label, dtype: int64

### Split the data into train/val/test

In [14]:
X = reviews_data["review"]
y = reviews_data["label"]

In [15]:
X_train, X_val1, y_train, y_val1 = train_test_split(X, y, test_size = 0.3, random_state = 17, stratify = reviews_data['label'])
X_val, X_test, y_val, y_test = train_test_split(X_val1, y_val1, test_size = 0.12, random_state = 18, stratify = y_val1)

In [16]:
print("X_train shape:",X_train.shape)
print("y_train shape:",y_train.shape)
print("X_val shape:",X_val.shape)
print("y_val shape:",y_val.shape)
print("X_test shape:",X_test.shape)
print("y_test shape:",y_test.shape)

X_train shape: (40880,)
y_train shape: (40880,)
X_val shape: (15418,)
y_val shape: (15418,)
X_test shape: (2103,)
y_test shape: (2103,)


In [17]:
reviews_data.set_index("review", inplace = True)
reviews_data.loc[X_train,"data_type"] = "train"
reviews_data.loc[X_val,"data_type"] = "val"
reviews_data.loc[X_test,"data_type"] = "test"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [18]:
reviews_data.reset_index(inplace = True)
reviews_data.head(3)

,review,recommended,label,data_type
0,Simple yet with great replayability. In my opi...,True,0,train
1,Its unique and worth a playthrough.,True,0,train
2,Great atmosphere. The gunplay can be a bit chu...,True,0,val


In [19]:
reviews_data.groupby(["recommended","label","data_type"]).count()

review
recommended label data_type        
False       1     test          284
                  train        4617
                  val          1835
True        0     test         2969
                  train       34234
                  val         14462

### Tokenization

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#do_lower_case = True

In [21]:
encoded_data_train = tokenizer.batch_encode_plus(X_train.to_list(),
                                                add_special_tokens = True, 
                                                return_attention_mask = True, 
                                                max_length = 256,
                                                padding = 'max_length', 
                                                truncation = True,  
                                                return_tensors = 'pt' )

encoded_data_val = tokenizer.batch_encode_plus(X_val.to_list(),
                                               add_special_tokens = True, 
                                               return_attention_mask = True, 
                                               max_length = 256,
                                               padding = 'max_length',   
                                               truncation = True, 
                                               return_tensors = 'pt')

encoded_data_test = tokenizer.batch_encode_plus(X_test.to_list(),
                                                add_special_tokens = True, 
                                                return_attention_mask = True, 
                                                max_length = 256, 
                                                padding = 'max_length',                                            
                                                truncation = True, 
                                                return_tensors = 'pt')

In [22]:
input_ids_train = encoded_data_train['input_ids']
#torch.tensor(encoded_data_train['input_ids'])
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.to_list())

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val.to_list())

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(y_test.to_list())

In [23]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [24]:
len(dataset_train),len(dataset_val),len(dataset_test)

(40880, 15418, 2103)

In [25]:
batch_size = 256

dataloader_train = DataLoader(dataset_train, 
                              sampler = RandomSampler(dataset_train), 
                              batch_size = batch_size)

dataloader_val = DataLoader(dataset_val, 
                            sampler = SequentialSampler(dataset_val), 
                            batch_size = batch_size)

dataloader_test = DataLoader(dataset_test, 
                             sampler = SequentialSampler(dataset_test), 
                             batch_size = batch_size)

### BERT model

In [26]:
# import BERT-base pretrained model
#bert = AutoModel.from_pretrained('bert-base-uncased')

bert = BertModel.from_pretrained("bert-base-uncased",
                                                     num_labels = len(label_dict),
                                                     output_attentions = False,
                                                     output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [29]:
class bertModel(nn.Module):

  def __init__(self, bert):

    super(bertModel, self).__init__()

    self.bert = bert
    self.dropout1 = nn.Dropout(0.1)
    self.relu =  nn.ReLU()
    #self.fc0 = nn.Linear(2, 768)
    self.fc1 = nn.Linear(self.bert.config.hidden_size, 512)
    #self.dropout2 = nn.Dropout(0.1)
    self.fc2 = nn.Linear(512, 2)
    self.softmax = nn.LogSoftmax(dim = 1)

  def forward(self, **inputs):

    _, x = self.bert(**inputs)

    #print("shape after self.bert:",x.shape)
    #print(x, x.reshape(-1,1))
    #print(self.bert.config.hidden_size)
    #x = x.view(x.size(0), -1)
 
    #print("after flatten",x.shape)
    #x = self.fc0(x)
    x = self.fc1(x)
    #print("shape after fc1:",x.shape)
    x = self.relu(x)
    x = self.dropout1(x)
    #print("shape after dropout of fc1:",x.shape)
    x = self.fc2(x)
    #print("final fc",x)
    x = self.softmax(x)

    return x

In [30]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [31]:
model = bertModel(bert)
#BERT_Arch #bertModel
model = model.to(device)

In [32]:
model

bertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [44]:
optimizer = AdamW(model.parameters(), lr = 0.00001)

epochs = 6
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = len(dataloader_train)*epochs)

In [45]:
#calculate class weights
class_wts = class_weight.compute_class_weight("balanced", np.unique(y_train), y_train)
class_wts = torch.FloatTensor(list(dict(enumerate(class_wts)).values())).to(device)
print("Class weights:",class_wts)

#class_weights = torch.tensor(class_wts, dtype = torch.float)
#class_weights = torch.FloatTensor(class_wts)
#class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight = class_wts)
criterion = criterion.to(device)

Class weights: tensor([0.5652, 4.3351], device='cuda:0')


In [46]:
#criterion = criterion.to(device)
#model = bertModel(bert)
#model.to(device)

train model function

In [47]:
seed_val = 24
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [50]:

def model_evaluate(dataloader, model, device, criterion):

  model.eval()

  total_loss, pred_loss = [], 0 

  with torch.no_grad():
    for idx, batch in enumerate(dataloader):

      data = tuple(b.to(device) for b in batch)
      inputs = {'input_ids':      data[0],
                'attention_mask': data[1],
                #'labels':         data[2],
                'return_dict' : False
              }   
      labels = data[2] 
      #data = [b.to(device) for b in batch]
      #inputs_ids, attn_masks, labels = data[0], data[1], data[2]
      
      #print("input shape:",data.shape)
      outputs = model(**inputs)
      #inputs_ids = inputs_ids, token_type_ids = None, attention_mask = attn_masks, labels = labels

      loss = criterion(outputs, labels)

      pred_loss += loss.item()
      total_loss.append(pred_loss)

  avg_pred_loss = np.mean(total_loss)

  return outputs, labels, avg_pred_loss


In [51]:

def model_train(model, device, criterion, scheduler, optimizer):

  model.train()
           
  total_train_loss, training_loss = [], 0 
  #progress_bar = tqdm(dataloader_train, desc = 'Epoch {:1d}'.format(epoch), leave = False, disable = False)

  for idx, batch in enumerate(dataloader_train):

    model.zero_grad()

    data = tuple(b.to(device) for b in batch)
    inputs = {'input_ids':      data[0],
              'attention_mask': data[1],
              #'labels':         data[2],
              'return_dict' : False
              }
    labels = data[2]
    outputs = model(**inputs)
    
    loss = criterion(outputs, labels)
    #loss_train_total += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    #update the weights
    optimizer.step()
    scheduler.step()
          
    #progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    training_loss += loss.item()
    total_train_loss.append(training_loss)

    if idx % 25 == 0:
      print('Batch: {}, Training Loss: {}'.format(idx, training_loss/10))
      training_loss = 0
      
  #avg training loss
  avg_train_loss = np.mean(total_train_loss)

  return avg_train_loss


In [55]:
#train_loss, val_loss = model_train(model, device, criterion, scheduler, optimizer, epochs)

epochs = 6
train_losses, val_losses = [], [] 

for epoch in range(1, epochs+1):

  print('Epoch: {} \n'.format(epoch))

  #train
  avg_train_loss = model_train(model, device, criterion, scheduler, optimizer)
  train_losses.append(avg_train_loss)
  
  #validation
  val_preds, target_labels, avg_pred_loss = model_evaluate(dataloader_val, model, device, criterion)
  val_losses.append(avg_pred_loss)
  
  print('End of Epoch {}, Avg. Training Loss: {}, Avg. validation Loss: {} \n'.format(epoch, avg_train_loss, avg_pred_loss))


Epoch: 1 

Batch: 0, Training Loss: 0.06706169843673707
Batch: 25, Training Loss: 1.666575825214386
Batch: 50, Training Loss: 1.6737629771232605
Batch: 75, Training Loss: 1.6768136382102967
Batch: 100, Training Loss: 1.6760462284088136
Batch: 125, Training Loss: 1.6587961971759797
Batch: 150, Training Loss: 1.6747236371040344
End of Epoch 1, Avg. Training Loss: 8.335414841398597, Avg. validation Loss: 20.584990633315726 

Epoch: 2 

Batch: 0, Training Loss: 0.0665098249912262
Batch: 25, Training Loss: 1.6797846257686615
Batch: 50, Training Loss: 1.6612519681453706
Batch: 75, Training Loss: 1.6818220555782317
Batch: 100, Training Loss: 1.6696908950805665
Batch: 125, Training Loss: 1.664349192380905
Batch: 150, Training Loss: 1.6789659202098846
End of Epoch 2, Avg. Training Loss: 8.352044053375721, Avg. validation Loss: 20.584990633315726 

Epoch: 3 

Batch: 0, Training Loss: 0.0665757417678833
Batch: 25, Training Loss: 1.6707032561302184
Batch: 50, Training Loss: 1.6741383910179137
Batc